### 1. Import data and revelant packages 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
X_train = pd.read_csv('/Users/admin/Desktop/Y3S2/BT4222/training_set.csv')
X_train.head()

,district,street,propertyType,remaining_lease,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,20,1487,8,-0.028291,37,11,5,1,122.714286,1.384051,103.900061,0.333809,1,0.056981
1,21,668,6,0.130119,40,1,8,0,-1.000000,1.358962,103.763673,-0.261162,2,0.145982
2,23,291,3,-0.030116,24,1,11,0,169.428571,1.383134,103.744511,-0.092774,1,0.073223
3,18,1734,4,-0.028981,26,4,14,0,163.857143,1.355347,103.960666,0.109292,1,0.052676
4,20,1932,4,-0.029067,37,11,5,0,122.714286,1.446986,103.799769,0.075614,1,0.083862


In [3]:
y_train = pd.read_csv('/Users/admin/Desktop/Y3S2/BT4222/training_labels.csv')
y_train.head()

,price
0,-0.268334
1,0.137525
2,-0.392790
3,-0.319220
4,-0.322286


In [4]:
X_test = pd.read_csv('/Users/admin/Desktop/Y3S2/BT4222/testing_set.csv')
X_test.head()

,district,street,propertyType,remaining_lease,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,27,1972,9,-0.026896,18,2,19,0,64.000000,1.425057,103.844262,-0.025418,2,0.071470
1,27,1988,2,-0.030849,18,2,19,0,64.000000,1.428767,103.832604,-0.407098,1,0.071470
2,20,141,4,-0.025876,37,11,5,0,122.714286,1.348742,103.711118,-0.025418,1,0.064371
3,19,1452,9,-0.026809,45,5,16,0,202.142857,1.401083,103.898665,-0.115225,3,0.083045
4,3,351,6,-0.026451,24,16,7,0,193.285714,1.303114,103.797330,-0.811229,2,0.091372


In [5]:
y_test = pd.read_csv('/Users/admin/Desktop/Y3S2/BT4222/testing_labels.csv')
y_test.head()

,price
0,0.014909
1,-0.415542
2,-0.291631
3,0.091544
4,-0.086249


In [6]:
df = pd.read_csv('/Users/admin/Desktop/Y3S2/BT4222/combined.csv')
df.head()

,district,street,propertyType,remaining_lease,price,school,hawkercentre,supermarkets,Bus Stops Nearby,crime_number,latitude,longitude,floor_area_sqm,floor_range,sentiment
0,5,2002,11,999.00,5500000.0,27,6,8,0,-1.0,1.282130,103.786879,524.3,0,0.129813
1,5,2002,11,999.00,5000000.0,27,6,8,0,-1.0,1.282130,103.786879,308.0,0,0.064396
2,5,2002,11,999.00,4750000.0,27,6,8,0,-1.0,1.282130,103.786879,314.0,0,0.064396
3,5,1322,15,999.00,2630000.0,27,6,8,2,-1.0,1.292047,103.768591,159.3,0,0.129813
4,4,375,6,84.92,2800000.0,11,3,2,0,-1.0,1.244208,103.827487,223.0,1,0.074654


### 2. Linear Regression

In [7]:
print(X_train.shape)
print(X_test.shape)

(119346, 14)
(13261, 14)


In [8]:
X_train.isnull().any()

district            False
street              False
propertyType        False
remaining_lease     False
school              False
hawkercentre        False
supermarkets        False
Bus Stops Nearby    False
crime_number        False
latitude            False
longitude           False
floor_area_sqm      False
floor_range         False
sentiment           False
dtype: bool

In [9]:
from sklearn.linear_model import LinearRegression

# Creates a LinearRegression object (Ordinary least squares Linear Regression)
lm = LinearRegression()

# Convert both y_train and y_test back to series
y_train = y_train.squeeze()
y_test = y_test.squeeze()

lm.fit(X_train, y_train)

LinearRegression()

### 2.1 Calculate Metrics for Base Model

In [10]:
# Compute MSE
def mse(y_predicted, y_true):
    return np.mean(np.square(y_predicted - y_true))

mse_training = mse(lm.predict(X_train), y_train)
mse_testing = mse(lm.predict(X_test), y_test)

print("training mse: {}, testing mse: {}".format(mse_training, mse_testing))
print("training rmse: {}, testing rmse: {}".format(np.sqrt(mse_training), np.sqrt(mse_testing)))

training mse: 0.22468369559844675, testing mse: 0.2010921166773819
training rmse: 0.4740081176503697, testing rmse: 0.4484329567252856


In [11]:
# Compute R2 and adjusted R2
print("training R2: {}, testing R2: {}".format(lm.score(X_train, y_train), lm.score(X_test, y_test)))
print("training adjusted R2: {}, testing adjusted R2: {}".format(
    1 - (1-lm.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1), 
    1 - (1-lm.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)))


training R2: 0.7753144217613501, testing R2: 0.7762345147014125
training adjusted R2: 0.7752880614853502, testing adjusted R2: 0.7759980118481602


In [12]:
# Calculate MAE
from sklearn.metrics import mean_absolute_error

print("training MAE: {}, testing MAE: {}".format(
    mean_absolute_error(y_train, lm.predict(X_train)),
    mean_absolute_error(y_test, lm.predict(X_test)),    
))

training MAE: 0.23867093561916156, testing MAE: 0.24168738953203142


### 2.2 Feature Cross

In [13]:
_, longitude_boundaries = np.histogram(X_train['longitude'])
_, latitude_boundaries = np.histogram(X_train['latitude'])

def assign_bucket(x, ranges):
    if x <= ranges[0]:
        return 0
    
    for i in range(1, len(ranges) - 1):
        if x <= ranges[i]:
            return i - 1
    return len(ranges) - 2

X_train['lgt_discrete'] = X_train['longitude'].apply(lambda x: assign_bucket(x, longitude_boundaries))
X_train['lat_discrete'] = X_train['latitude'].apply(lambda x: assign_bucket(x, latitude_boundaries))
X_test['lgt_discrete'] = X_test['longitude'].apply(lambda x: assign_bucket(x, longitude_boundaries))
X_test['lat_discrete'] = X_test['latitude'].apply(lambda x: assign_bucket(x, latitude_boundaries))

In [14]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories='auto')
features_lat_lgn_train = enc.fit_transform(X_train[['lgt_discrete', 'lat_discrete']]) # fit and transform TRAIN
features_lat_lgn_test = enc.transform(X_test[['lgt_discrete', 'lat_discrete']]) # transform only TEST

### 2.3 Error Metrics and Coefficients

In [15]:
features_to_drop = ['longitude', 'latitude', 'lgt_discrete', 'lat_discrete']
OHE_train = np.hstack((X_train.drop(features_to_drop, axis=1).values, features_lat_lgn_train.toarray()))
OHE_test = np.hstack((X_test.drop(features_to_drop, axis=1).values, features_lat_lgn_test.toarray()))

lm.fit(OHE_train, y_train)

mse_training = mse(lm.predict(OHE_train), y_train)
mse_testing = mse(lm.predict(OHE_test), y_test)

print("training mse: {}, testing mse: {}".format(mse_training, mse_testing))
print("training rmse: {}, testing rmse: {}".format(np.sqrt(mse_training), np.sqrt(mse_testing)))

training mse: 0.21053513509999797, testing mse: 0.1875863329615638
training rmse: 0.45884107826130605, testing rmse: 0.43311237913682843


In [16]:
# Compute R2 and adjusted R2
print("training R2: {}, testing R2: {}".format(lm.score(OHE_train, y_train), lm.score(OHE_test, y_test)))
print("training adjusted R2: {}, testing adjusted R2: {}".format(
    1 - (1-lm.score(OHE_train, y_train))*(len(y_train)-1)/(len(y_train)-OHE_train.shape[1]-1), 
    1 - (1-lm.score(OHE_test, y_test))*(len(y_test)-1)/(len(y_test)-OHE_test.shape[1]-1)))


training R2: 0.7894631008115658, testing R2: 0.7912630911440999
training adjusted R2: 0.7894066343680598, testing adjusted R2: 0.7907581333966408


In [17]:
# Calculate MAE
print("training MAE: {}, testing MAE: {}".format(
    mean_absolute_error(y_train, lm.predict(OHE_train)),
    mean_absolute_error(y_test, lm.predict(OHE_test)),    
))

training MAE: 0.22905099602076384, testing MAE: 0.2319691885977825


In [18]:
print(OHE_train.shape)
print()
print(y_train.shape)

(119346, 32)

(119346,)


In [19]:
pd.DataFrame([x for x in zip(list(X_train.drop(features_to_drop, axis=1).columns) \
              + [str(x) for x in range(31)], lm.coef_)], columns=['features', 'coefficients'])

,features,coefficients
0,district,-0.025432
1,street,-0.000067
2,propertyType,0.009682
3,remaining_lease,0.003093
4,school,0.004341
5,hawkercentre,-0.037834
6,supermarkets,0.000023
7,Bus Stops Nearby,0.017769
8,crime_number,-0.000578
9,floor_area_sqm,0.779610


### 3. Polynomial Regression and Error Metrics

In [20]:
from sklearn.preprocessing import PolynomialFeatures
pre_process = PolynomialFeatures(degree=2)

# Transform with OHE datasets
X_poly = pre_process.fit_transform(OHE_train)
X_test_poly = pre_process.transform(OHE_test)

poly_model = LinearRegression()
poly_model.fit(X_poly, y_train)

mse_training = mse(poly_model.predict(X_poly), y_train)
mse_testing = mse(poly_model.predict(X_test_poly), y_test)

print("training mse: {}, testing mse: {}".format(mse_training, mse_testing))
print("training rmse: {}, testing rmse: {}".format(np.sqrt(mse_training), np.sqrt(mse_testing)))

training mse: 0.100890205346986, testing mse: 0.072010661382542
training rmse: 0.3176321856282609, testing rmse: 0.26834802287801934


In [21]:
# Compute R2 and adjusted R2
print("training R2: {}, testing R2: {}".format(poly_model.score(X_poly, y_train), 
                                               poly_model.score(X_test_poly, y_test)))
print("training adjusted R2: {}, testing adjusted R2: {}".format(
    1 - (1-poly_model.score(X_poly, y_train))*(len(y_train)-1)/(len(y_train)-X_poly.shape[1]-1), 
    1 - (1-poly_model.score(X_test_poly, y_test))*(len(y_test)-1)/(len(y_test)-X_test_poly.shape[1]-1)))


training R2: 0.8991089492870157, testing R2: 0.9198700533010541
training adjusted R2: 0.8986324551510211, testing adjusted R2: 0.9163301761376468


In [22]:
# Calculate MAE
print("training MAE: {}, testing MAE: {}".format(
    mean_absolute_error(y_train, poly_model.predict(X_poly)),
    mean_absolute_error(y_test, poly_model.predict(X_test_poly)),    
))

training MAE: 0.12832259602162782, testing MAE: 0.1295353707070946


In [23]:
pd.DataFrame([x for x in zip(list(X_train.drop(features_to_drop, axis=1).columns) \
              + [str(x) for x in range(31)], poly_model.coef_)], columns=['features', 'coefficients'])

,features,coefficients
0,district,-6.779749e-09
1,street,2.107117e-01
2,propertyType,2.999848e-05
3,remaining_lease,5.565384e-02
4,school,2.067873e+00
5,hawkercentre,6.232552e-02
6,supermarkets,-2.754534e-01
7,Bus Stops Nearby,-2.976763e-01
8,crime_number,-1.664782e-01
9,floor_area_sqm,7.911236e-04
